In [1]:
import nemo.collections.asr as nemo_asr
asr_model = nemo_asr.models.ASRModel.from_pretrained(model_name="nvidia/parakeet-tdt-0.6b-v2")

/mnt/fr20tb/kyudan/miniforge3/envs/dataset/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/mnt/fr20tb/kyudan/miniforge3/envs/dataset/lib/python3.10/site-packages/lightning/fabric/__init__.py:41: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
/mnt/fr20tb/kyudan/miniforge3/envs/dataset/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/fr20tb/kyudan/miniforge3/envs/datas

[NeMo I 2025-12-02 02:04:04 mixins:184] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2025-12-02 02:04:04 modelPT:188] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_lhotse: true
    skip_missing_manifest_entries: true
    input_cfg: null
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    num_workers: 2
    pin_memory: true
    max_duration: 40.0
    min_duration: 0.1
    text_field: answer
    batch_duration: null
    use_bucketing: true
    bucket_duration_bins: null
    bucket_batch_size: null
    num_buckets: 30
    bucket_buffer_size: 20000
    shuffle_buffer_size: 10000
    
[NeMo W 2025-12-02 02:04:04 modelPT:195] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    use_

[NeMo I 2025-12-02 02:04:04 features:306] PADDING: 0
[NeMo I 2025-12-02 02:04:09 rnnt_models:226] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-12-02 02:04:09 rnnt_models:226] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}


[NeMo W 2025-12-02 02:04:09 label_looping_base:109] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: No `cuda-python` module. Please do `pip install cuda-python>=12.3`


[NeMo I 2025-12-02 02:04:09 rnnt_models:226] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}


[NeMo W 2025-12-02 02:04:09 label_looping_base:109] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: No `cuda-python` module. Please do `pip install cuda-python>=12.3`


[NeMo I 2025-12-02 02:04:11 save_restore_connector:282] Model EncDecRNNTBPEModel was successfully restored from /mnt/ddn/kyudan/.cache/huggingface/hub/models--nvidia--parakeet-tdt-0.6b-v2/snapshots/48b630d20b000e5ad3735e5378a2d9bde3f80826/parakeet-tdt-0.6b-v2.nemo.


In [2]:
!wget https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav

--2025-12-02 02:04:15--  https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
Resolving dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)... 52.219.94.146, 3.5.133.198, 52.219.177.162, ...
Connecting to dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)|52.219.94.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 237964 (232K) [audio/wav]
Saving to: ‘2086-149220-0033.wav’

2086-149220-0033.wa 100%[===================>] 232.39K   254KB/s    in 0.9s    

2025-12-02 02:04:17 (254 KB/s) - ‘2086-149220-0033.wav’ saved [237964/237964]



In [3]:
output = asr_model.transcribe(['2086-149220-0033.wav'])
print(output[0].text)

Transcribing: 100%|██████████| 1/1 [00:02<00:00,  2.34s/it]

Well, I don't wish to see it any more, observed Phebe, turning away her eyes. It is certainly very like the old portrait.


In [ ]:
output[0]

[Hypothesis(score=4257.033203125, y_sequence=tensor([735, 839,  34, 200, 844, 821,   7, 379,  20, 253,  51, 296, 268, 839,
         601, 826,  12, 548, 206, 546, 430, 839,   1,  92, 466,   3, 374, 427,
          60, 833,  27, 841, 245,  59,  16, 611, 177,  83, 297, 117,   5,   8,
          82,  24, 208, 148,  35, 841]), text="Well, I don't wish to see it any more, observed Phebe, turning away her eyes. It is certainly very like the old portrait.", dec_out=None, dec_state=None, timestamp=[], alignments=None, frame_confidence=None, token_confidence=None, word_confidence=None, length=0, y=None, lm_state=None, lm_scores=None, ngram_lm_state=None, tokens=None, last_token=None, token_duration=None, last_frame=None)]

In [6]:
from models import separate_fast, dnsmos, whisper_asr, silero_vad
asr_model_whisper = whisper_asr.load_asr_model(
            "large-v3",
            "cuda",
            compute_type="float16",
            threads=4,
            language="en", 
            
            )

In [7]:
transcribe = asr_model_whisper.transcribe('2086-149220-0033.wav')

TypeError: VadFreeFasterWhisperPipeline.transcribe() missing 1 required positional argument: 'vad_segments'

In [3]:
!export CUDA_VISIBLE_DEVICES=0
from nemo.collections.speechlm2.models import SALM

model = SALM.from_pretrained('nvidia/canary-qwen-2.5b')


[NeMo I 2025-12-02 04:19:23 auto_tokenizer:234] 1 special tokens added, resize your model accordingly.
[NeMo I 2025-12-02 04:20:01 lora:30] LoRA adapter installed: LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='Qwen/Qwen3-1.7B', revision=None, inference_mode=False, r=128, target_modules=['q_proj', 'v_proj'], exclude_modules=None, lora_alpha=256, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)
[NeMo I 2025-12-02 04:20:01 features:306] PADDING: 0


In [2]:
answer_ids = model.generate(
    prompts=[
        [{"role": "user", "content": f"Transcribe the following: {model.audio_locator_tag}", "audio": ["2086-149220-0033.wav"]}]
    ],
    max_new_tokens=128,
)
print(model.tokenizer.ids_to_text(answer_ids[0].cpu()))


`generation_config` default values have been modified to match model-specific defaults: {'bos_token_id': 151643}. If this is not desired, please set these values explicitly.


Well, I don't wish to see it any more, observed Phoebe, turning away her eyes. It is certainly very like the old portrait.
